# Especialização em Ciência de Dados - PUC-Rio
# Machine Learning - Prof. Tatiana Escovedo
## Seleção de Atributos

In [ ]:
# Imports
import pandas as pd
import numpy as np
from numpy import set_printoptions
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier

## Carga do dataset

In [ ]:
# Carrega arquivo csv usando Pandas usando uma URL

# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# Informa o cabeçalho das colunas
colunas = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

# Lê o arquivo utilizando as colunas informadas
dataset = pd.read_csv(url, names=colunas, skiprows=0, delimiter=',')

## Particionamento em conjuntos de treino e teste

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# separando os atributos e a classe do dataset
array = dataset.values
X = array[:,0:8]
Y = array[:,8]

# definindo o tamanho do conjunto de teste
test_size = 0.20 

# A semente (seed) pode ser qualquer número, e garante que os resultados possam ser reproduzidos de forma idêntica toda vez que o script for rodado. 
# Isto é muito importante quando trabalhamos com modelos ou métodos que utilizam de algum tipo de aleatoriedade.
seed = 7

# particionando em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

## Validação Cruzada e Métricas de avaliação

* Validação Cruzada: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
* Métricas de avaliação: http://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
# definindo a métrica de avaliação dos algoritmos
scoring = 'accuracy'
# scoring = 'roc_auc'

kfold = KFold(n_splits=10)

## Seleção de Atributos

### Seleção Univariada
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

A função **SelectKBest()** pode ser usada com diversos testes estatísticos para selecionar os atributos. Vamos usar o teste qui-quadrado e selecionar os 4 melhores atributos que podem ser usados como variáveis preditoras.

> O teste estatístico qui-quadrado é aplicado a dados categóricos para avaliar o quão provável é que qualquer diferença absoluta observada aconteça ao **acaso**, verificando se a frequência absoluta **observada** de uma variável é significativamente diferente da distribuição de frequência absoluta **esperada**. *Para saber mais: https://pt.wikipedia.org/wiki/Teste_qui-quadrado_de_Pearson*

In [ ]:
# Função para seleção de atributos
best_var = SelectKBest(score_func=chi2, k=4)

# Executa a função de pontuação em (X_train, Y_train) e obtém os atributos selecionados
fit = best_var.fit(X_train, Y_train)

# Reduz X para os atributos selecionados
features = fit.transform(X_train)

# Resultados
print('\nNúmero original de atributos:', X.shape[1])
print('\nNúmero reduzido de atributos:', features.shape[1])

# Exibe os atributos orginais
print("\nAtributos Originais:", dataset.columns[0:8])

# Exibe as pontuações de cada atributos e os 4 escolhidas (com as pontuações mais altas): plas, test, mass e age.
# (Basta mapear manualmente o índice dos nomes dos respectivos atributos)
set_printoptions(precision=3) # 3 casas decimais
print(fit.scores_)


Número original de atributos: 8

Número reduzido de atributos: 4

Atributos Originais: Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'], dtype='object')
[  78.494 1070.85    13.976   67.539 1237.401  116.909    5.281  119.867]


### Eliminação Recursiva de Atributos
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

Iremos aplicar a técnica de eliminação recursiva de atributos com um algoritmo de Regressão Logística (poderia ser qualquer classificador) para selecionar as 4 melhores variáveis preditoras.

In [ ]:
# Criação do modelo
modelo = LogisticRegression(max_iter=200)

# Eliminação Recursiva de Variáveis
rfe = RFE(modelo, 4)
fit = rfe.fit(X_train, Y_train)

# Print dos resultados
print("Atributos preditores:", dataset.columns[0:8])

# Exibe os atributos selecionados (marcados como True em "Atributos Selecionados" 
# e com valor 1 em "Ranking dos Atributos"): preg, plas, mass e pedi.
# (Basta mapear manualmente o índice dos nomes dos respectivos atributos)
print("\nAtributos selecionados: %s" % fit.support_)
print("\nRanking de atributos: %s" % fit.ranking_)
print("\nQtd de melhores Atributos: %d" % fit.n_features_)

Atributos preditores: Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'], dtype='object')

Atributos selecionados: [ True  True False False False  True  True False]

Ranking de atributos: [1 1 2 4 5 1 1 3]

Qtd de melhores Atributos: 4


### Importância de Atributos com ExtraTrees
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html

Iremos construir um classificador ExtraTreesClassifier.

In [ ]:
# Criação do modelo para seleção de atributos
modelo = ExtraTreesClassifier(n_estimators=100)
modelo.fit(X_train, Y_train)

# Exibe os atributos orginais
print("\nAtributos Originais:", dataset.columns[0:8])

# Exibe a pontuação de importância para cada atributo (quanto maior a pontuação, mais importante é o atributo). 
# Atributos selecionados: plas, mass, pedi, age.
print(modelo.feature_importances_)


Atributos Originais: Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'], dtype='object')
[0.11  0.222 0.101 0.082 0.079 0.147 0.12  0.139]
